This is an example script for the PhotoReceptor model. Please run each cell in sequence.

If you want to change how many iterations are run, you can do so in the second cell.
- nproposals: number of proposals evaluated in parallel in the Generalized Metropolis Hastings algorithm
- niterations: number of total iterations

The values given below (1000 proposals for 100 iterations) will take about 5 minutes to run on an 8-core machine.

Remember that you can remove output from the cells by selecting "Cell/All Output/Clear". And if the Kernel hangs for some reason, then you can choose "Kernel/Restart" from the menu.

In [3]:
###Load the PyPlot package
import PyPlot
println("PyPlot package loaded successfully")

###You will need to run this cell only once, unless some workers crash and shut down
NPROCS = 17 #make this the number of processors on your machine. For JuliaBox, it is 16
if nprocs() < NPROCS
    addprocs(NPROCS-nprocs())
end
println("Number of parallel processes: ",nprocs())

import GeneralizedMetropolisHastings
import GMHPhotoReceptor

@everywhere using GeneralizedMetropolisHastings
@everywhere using GMHPhotoReceptor
println("GMH modules loaded successfully")

INFO: Recompiling stale cache file /home/juser/.julia/lib/v0.4/MacroTools.ji for module MacroTools.
  This is likely because module Compat does not support  precompilation but is imported by a module that does.
INFO: Recompiling stale cache file /opt/julia_packages/.julia/lib/v0.4/MacroTools.ji for module MacroTools.
INFO: Recompiling stale cache file /home/juser/.julia/lib/v0.4/PyPlot.ji for module PyPlot.
INFO: Recompiling stale cache file /home/juser/.julia/lib/v0.4/PyCall.ji for module PyCall.
  This is likely because module JSON does not support  precompilation but is imported by a module that does.
INFO: Recompiling stale cache file /opt/julia_packages/.julia/lib/v0.4/PyPlot.ji for module PyPlot.
INFO: Recompiling stale cache file /home/juser/.julia/lib/v0.4/Colors.ji for module Colors.
  This is likely because module Compat does not support  precompilation but is imported by a module that does.
INFO: Recompiling stale cache file /opt/julia_packages/.julia/lib/v0.4/Colors.ji for 

LoadError: LoadError: __precompile__(true) but require failed to create a precompiled cache file
while loading In[3], in expression starting on line 2

In [ ]:
#Standard M-H for nproposals == 1
#Generalized M-H for nproposals > 1
nproposals = 1024

#MCMC iteration specifications
nburnin = 100
niterations = 100
ntunerperiod = 10

###Values of the model
numvilli1 = 30000

#specify the values that determine the priors on the parameters
latencylocation = (2.0,3.5) #uniform distribution with (low,high) values
latencyscale = (0.2,0.7) #uniform distribution with (low,high) values
refractorylocation = (4.0,6.0) #uniform distribution with (low,high) values
refractoryscale = (1.5,2.5) #uniform distribution with (low,high) values
bumpamplitude = (3.0,5.0) #uniform distribution with (low,high) values
bumpshape = (log(3.0),0.1) #lognormal distribution with (location,scale) 
bumpscale = (log(2.5),0.1) #lognormal distribution with (location,scale)

photons1 = photonsequence("../data/naturallight.jld")
current1 = lightinducedcurrent("../data/naturallight.jld")

modelpolicy1 = policy(:photoreceptor;bump = :sample) #7-parameter model with latency, refractory and bump parameters
params1 = parameters(:photoreceptor,modelpolicy1,latencylocation,latencyscale,refractorylocation,refractoryscale,
                                                bumpamplitude,bumpshape,bumpscale)

####Variance for the noise model, estimated from previous runs
variance1 = [3600.0]

println("==========================================")
println("Simulation parameters defined successfully")
println("==========================================")

In [ ]:
###Create a PhotoReceptor model
model1 = model(:photoreceptor,params1,photons1,current1,variance1,numvilli1,modelpolicy1)

###Show the model
println("==========================")
println("Model defined successfully")
println("==========================")
show(model1)

In [ ]:
###Plot the measurement data
PyPlot.figure("PhotoReceptor-Measurement") ; PyPlot.clf()
PyPlot.plot(dataindex(model1),measurements(model1);label="measured",linewidth=2)
PyPlot.xlabel("Time")
PyPlot.ylabel("Current (nA)")
PyPlot.xlim(dataindex(model1)[1],dataindex(model1)[end])
PyPlot.title("Light-Induced Current")
PyPlot.grid("on")
PyPlot.legend(loc="upper right",fancybox="true")

In [ ]:
###Evaluate the model for a set of parameter values and plot the fit
###You can execute this cell as many times as you like
###Parameter values are drawn from the prior distributions on the parameters
numevaluations = 100
paramvals = GeneralizedMetropolisHastings.initvalues!(trait(:initialize,:prior),params1,zeros(Float64,length(params1)))
println("Evaluating the model $numevaluations times")
evaldata = evaluate!(model1,paramvals,numevaluations)
logposteriorvals = zeros(numevaluations)
for i=1:numevaluations
    logposteriorvals[i] = loglikelihood(model1,evaldata[:,i])+logprior(params1,paramvals,Float64)
end
meanevaldata = mean(evaldata,2)

PyPlot.figure("PhotoReceptor-Evaluated")
PyPlot.plot(dataindex(model1),evaldata)
PyPlot.plot(dataindex(model1),meanevaldata;label="mean",linewidth=1,color="yellow")
PyPlot.xlim(dataindex(model1)[1],dataindex(model1)[end])
PyPlot.legend(loc="upper right",fancybox="true")
PyPlot.title("Variability in Light-Induced Current for 100 Model Evaluations")

PyPlot.figure("PhotoReceptor-Mean-Measured")
PyPlot.plot(dataindex(model1),measurements(model1);label="measured",linewidth=1,color="magenta")
PyPlot.plot(dataindex(model1),meanevaldata;label="mean",linewidth=1,color="yellow")
PyPlot.xlim(dataindex(model1)[1],dataindex(model1)[end])
PyPlot.legend(loc="upper right",fancybox="true")
PyPlot.title("Comparison of Mean vs Measured Light-Induced Current")

println("Evaluation paramater values: ")
display(paramvals)
println("Mean Log-Posterior for these parameters: ",mean(logposteriorvals))

In [ ]:
###Create a Metropolis sampler with a Normal proposal density
propcov = [0.1,0.01,0.1,0.01,0.1,0.01,0.01]
sampler1 = sampler(:mh,:normal,1.0,propcov)
println("============================")
println("Sampler defined successfully")
println("============================")
show(sampler1)

###Create a tuner, either one that scales the stepsize of the proposal density or one that monitors acceptance rates
#tuner1 = tuner(:scale,ntunerperiod,0.5,:erf)
tuner1 = tuner(:monitor,ntunerperiod)
println("==========================")
println("Tuner defined successfully")
println("==========================")
show(tuner1)

###Create a Generalized Metropolis-Hastings runner (which will default to Standard MH when nproposals=1)
runnerpolicy1 = policy(:mh,nproposals;initialize=:prior)
runner1 = runner(runnerpolicy1,niterations,nproposals;numburnin = nburnin)
println("===========================")
println("Runner defined successfully")
println("===========================")
show(runner1)

In [ ]:
###Run the MCMC (can take quite a bit of time)
println("=======================")
println("Run the MCMC simulation")
println("=======================")
chain1 = run!(runner1,model1,sampler1,tuner1)
println("=========================")
println("Completed MCMC simulation")
println("=========================")

In [ ]:
###Show the result of the simulations
show(chain1)
nparas = numparas(model1)
meanparamvals = mean(samples(chain1),2)
stdparamvals = std(samples(chain1),2)

println("Results of the MCMC simulation:")
for i=1:nparas
    println("mean $(parameters(model1)[i].key): $(meanparamvals[i])")
    println("std $(parameters(model1)[i].key): $(stdparamvals[i])")
end

In [ ]:
println("================")
println("Plotting results")
println("================")

###Plot the average model results in the data window
PyPlot.figure()
modeldata = evaluate!(model1,vec(meanparamvals))
PyPlot.plot(dataindex(model1),measurements(model1);label="measured",linewidth=4,color="yellow")
PyPlot.plot(dataindex(model1),modeldata;label="mean parameters",linewidth=1,color="black")
PyPlot.xlim(dataindex(model1)[1],dataindex(model1)[end])
PyPlot.legend(loc="upper right",fancybox="true")
PyPlot.title("Model Light-Induced Current for Mean MCMC Parameter Values")

###Plot the logposterior values across samples
PyPlot.figure()
PyPlot.plot(1:numsamples(chain1),logposterior(chain1,model1))
PyPlot.title("Log-Posterior values across samples")
PyPlot.xlabel("Samples")
PyPlot.ylabel("Log-Posterior")

In [ ]:
###Plot the histograms of latency and refractory parameter values
for i=1:4
    PyPlot.subplot(220 + i)
    bins = meanparamvals[i]-0.2:0.01:meanparamvals[i]+0.2
    h = PyPlot.plt[:hist](sub(samples(chain1),i,:)',bins)
    PyPlot.grid("on")
    PyPlot.title("$(parameters(model1)[i].key)")
    if i > 2
        PyPlot.xlabel("Values")
    end
    if i == 1 || i == 3
        PyPlot.ylabel("Samples")
    end
    PyPlot.xlim(meanparamvals[i]-0.2,meanparamvals[i]+0.2)
end

In [ ]:
###Plot the histograms of bump shape parameter values
for i=1:3
    PyPlot.subplot(220 + i)
    bins = meanparamvals[i+4]-0.2:0.01:meanparamvals[i+4]+0.2
    h = PyPlot.plt[:hist](sub(samples(chain1),i+4,:)',bins)
    PyPlot.grid("on")
    PyPlot.title("$(parameters(model1)[i+4].key)")
    if i > 2
        PyPlot.xlabel("Values")
    end
    if i == 1 || i == 3
        PyPlot.ylabel("Samples")
    end
    PyPlot.xlim(meanparamvals[i+4]-0.2,meanparamvals[i+4]+0.2)
end

###Plot the bump shape
PyPlot.subplot(224)
fixedbump = GMHPhotoReceptor.bump(GMHPhotoReceptor.fixedbumpvalues(modelpolicy1.fixedbump)...)
meanbump = GMHPhotoReceptor.bump(meanparamvals[5],meanparamvals[6],meanparamvals[7],modelpolicy1.fixedbump.cutoff)
PyPlot.plot(fixedbump;label="Fixed")
PyPlot.plot(meanbump;label="Mean")
PyPlot.title("Bump")
PyPlot.xlabel("Time")
PyPlot.legend(loc="upper right",fancybox="true")

println("Length of bump with fixed values: $(length(fixedbump))")
println("Length of average fitted bump: $(length(meanbump))")

In [ ]:
###Only run this box if you want to shut down all worker processes
println("Pre processes running: ",procs())
for i in workers()
    if i != 1
        rmprocs(i)
    end
end
println("Post processes running: ",procs())